In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature
import csv
from datetime import datetime
import metpy
import metpy.calc as mpcalc
from metpy.units import units
from time import sleep
import pygrib
import glob

In [2]:
hi_files = sorted(glob.glob("/glade/work/lgetker/daily_max_hi/*"))
t_max_files = sorted(glob.glob("/glade/work/lgetker/daily_max_t/*"))

In [3]:
hi_ds = xr.open_dataset(hi_files[0])
print(hi_ds)
lats = hi_ds['lat'].values
lons = hi_ds['lon'].values
temp_ds = xr.open_dataset(t_max_files[0])
print(temp_ds)

<xarray.Dataset>
Dimensions:  (time: 153, lat: 259, lon: 559)
Coordinates:
  * time     (time) datetime64[ns] 1940-05-01T23:00:00 ... 1940-09-30T23:00:00
  * lat      (lat) float64 74.75 74.5 74.25 74.0 73.75 ... 11.0 10.75 10.5 10.25
  * lon      (lon) float64 180.2 180.5 180.8 181.0 ... 319.0 319.2 319.5 319.8
Data variables:
    2hi_max  (time, lat, lon) float64 ...
<xarray.Dataset>
Dimensions:  (time: 153, lat: 259, lon: 559)
Coordinates:
  * time     (time) datetime64[ns] 1940-05-01T23:00:00 ... 1940-09-30T23:00:00
  * lat      (lat) float64 74.75 74.5 74.25 74.0 73.75 ... 11.0 10.75 10.5 10.25
  * lon      (lon) float64 180.2 180.5 180.8 181.0 ... 319.0 319.2 319.5 319.8
Data variables:
    2t_max   (time, lat, lon) float64 ...


In [4]:
print(lats.shape)
print(lons.shape)

(259,)
(559,)


In [5]:
for i in range(len(hi_files)):
    print(hi_files[i])
    hi_ds = xr.open_dataset(hi_files[i])
    temp_ds = xr.open_dataset(t_max_files[i])
    times = hi_ds['time'].values
    new_year = np.zeros([len(times), len(lats), len(lons)])
    diff = np.zeros([len(times), len(lats), len(lons)])
    for j in range(len(times)):
        print(times[j])
        hi_ds_time = hi_ds.sel(time = times[j])
        temp_ds_time = temp_ds.sel(time = times[j])
        hi = hi_ds_time['2hi_max'].values
        temp = temp_ds_time['2t_max'].values
        temp = (9/5) * (temp - 273) + 32
        new_values = np.zeros_like(hi)
        
        new_values = np.where(np.greater(hi, temp), hi, temp)
        diff[j,:,:] = np.where(np.greater(hi, temp), 0, 1)
        new_year[j, :, :] = new_values
    # ds = xr.Dataset(
    #     data_vars=dict(
    #         temp_or_hi=(["time", "lat", "lon"], new_year)
    #     ),
    #     coords=dict(
    #         lon=lons,
    #         lat=lats,
    #         time=times,
    #     ),
    #     attrs=dict(description="Arrays containing either daily max heat index or daily max temperature, whichever is higher.", units="degC"),
    # )
    # year = pd.Timestamp(times[i]).year
    # ds.to_netcdf("/glade/work/lgetker/daily_max_hi_or_temp/" + str(year) + "_daily_max_hi_or_temp_MJJAS_NA.nc", mode = 'w', format = 'NETCDF4')
    
print(ds)

/glade/work/lgetker/daily_max_hi/1940_daily_maxes_MJJAS_NA.nc
1940-05-01T23:00:00.000000000
1940-05-02T23:00:00.000000000
1940-05-03T23:00:00.000000000
1940-05-04T23:00:00.000000000
1940-05-05T23:00:00.000000000
1940-05-06T23:00:00.000000000
1940-05-07T23:00:00.000000000
1940-05-08T23:00:00.000000000
1940-05-09T23:00:00.000000000
1940-05-10T23:00:00.000000000
1940-05-11T23:00:00.000000000
1940-05-12T23:00:00.000000000
1940-05-13T23:00:00.000000000
1940-05-14T23:00:00.000000000
1940-05-15T23:00:00.000000000
1940-05-16T23:00:00.000000000
1940-05-17T23:00:00.000000000
1940-05-18T23:00:00.000000000
1940-05-19T23:00:00.000000000
1940-05-20T23:00:00.000000000
1940-05-21T23:00:00.000000000
1940-05-22T23:00:00.000000000
1940-05-23T23:00:00.000000000
1940-05-24T23:00:00.000000000
1940-05-25T23:00:00.000000000
1940-05-26T23:00:00.000000000
1940-05-27T23:00:00.000000000
1940-05-28T23:00:00.000000000
1940-05-29T23:00:00.000000000
1940-05-30T23:00:00.000000000
1940-05-31T23:00:00.000000000
1940-06-

NameError: name 'ds' is not defined

In [ ]:
fig = plt.figure(figsize=(16,16))
ax = plt.axes(projection=ccrs.Mercator(central_longitude = 0))
ax.add_feature(cfeature.STATES, edgecolor='black')  #Add US states
ax.add_feature(cfeature.COASTLINE, edgecolor='black')  #Add coastlines
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
ax.set_extent([-130, -60, 20, 50], crs=ccrs.PlateCarree())
cs = ax.contourf(lons, lats, diff[75,:,:], cmap = "bwr", transform=ccrs.PlateCarree())#, levels = np.arange(0,2,1))
cax = fig.add_axes([ax.get_position().x1+0.05,ax.get_position().y0,0.02,ax.get_position().height])  #You can change the numbers to move the colorbar.
plt.colorbar(cs, cax = cax)

In [ ]:
fig = plt.figure(figsize=(16,16))
ax = plt.axes(projection=ccrs.Mercator(central_longitude = 0))
ax.add_feature(cfeature.STATES, edgecolor='black')  #Add US states
ax.add_feature(cfeature.COASTLINE, edgecolor='black')  #Add coastlines
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
ax.set_extent([-130, -60, 20, 50], crs=ccrs.PlateCarree())
cs = ax.contourf(lons, lats, new_year[75,:,:], cmap = "jet", transform=ccrs.PlateCarree(), levels = np.arange(15,120,1))
ax.contour(lons, lats, diff[75,:,:],colors = 'black',transform=ccrs.PlateCarree())#, levels = np.arange(0,2,1))
cax = fig.add_axes([ax.get_position().x1+0.05,ax.get_position().y0,0.02,ax.get_position().height])  #You can change the numbers to move the colorbar.
plt.colorbar(cs, cax = cax)

In [ ]:
print(times[75])